# 3D Stereo Depth Reconstruction Using Numerical Optimization

**MAT353 Numerical Analysis Course Project**

**Student:** Farah Alhasan

**Student ID:** 21120205709

**GitHub:** https://github.com/FarahBebs/3D-STEREO-DEPTH-RECONSTRUCTION

---

## Project Overview

This notebook implements stereo depth reconstruction using:
- **Block Matching**: Traditional SSD-based disparity computation
- **Levenberg-Marquardt Optimization**: Numerical optimization for disparity refinement
- **Depth Reconstruction**: Converting disparity to 3D depth
- **Point Cloud Generation**: 3D reconstruction from depth data

In [ ]:
# Install required packages
!pip install numpy scipy opencv-python matplotlib -q

print("✓ All packages installed successfully!")

## 2. Import Libraries

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import cv2
from mpl_toolkits.mplot3d import Axes3D

# Set matplotlib parameters for better visualization
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print("✓ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 3. Synthetic Stereo Data Generation

Generate synthetic stereo pairs with known ground truth disparity for validation.

In [ ]:
def generate_synthetic_stereo_pair(width=640, height=480, num_objects=20):
    """
    Generate synthetic stereo pair with ground truth disparity.
    
    Parameters:
    - width, height: Image dimensions
    - num_objects: Number of objects in scene
    
    Returns:
    - left_image, right_image: Stereo pair
    - gt_disparity: Ground truth disparity map
    - camera_params: Camera parameters
    """
    np.random.seed(42)
    
    # Create depth map with multiple planes
    depth_map = np.ones((height, width)) * 100.0
    
    # Add random rectangles at different depths
    for i in range(num_objects):
        x = np.random.randint(50, width - 100)
        y = np.random.randint(50, height - 100)
        w = np.random.randint(40, 120)
        h = np.random.randint(40, 120)
        depth = np.random.uniform(20, 80)
        
        depth_map[y:y+h, x:x+w] = depth
    
    # Apply Gaussian smoothing for realistic depth transitions
    depth_map = cv2.GaussianBlur(depth_map, (15, 15), 5)
    
    # Camera parameters
    focal_length = 500.0  # pixels
    baseline = 10.0  # cm
    
    # Compute ground truth disparity: d = (f * B) / Z
    gt_disparity = (focal_length * baseline) / depth_map
    gt_disparity = np.clip(gt_disparity, 0, 64)
    
    # Create textured left image
    left_image = np.random.randint(0, 256, (height, width), dtype=np.uint8)
    left_image = cv2.GaussianBlur(left_image, (5, 5), 1.5)
    
    # Add some structured patterns
    for i in range(0, height, 40):
        cv2.line(left_image, (0, i), (width, i), int(np.random.randint(100, 200)), 2)
    for j in range(0, width, 40):
        cv2.line(left_image, (j, 0), (j, height), int(np.random.randint(100, 200)), 2)
    
    # Generate right image by shifting according to disparity
    right_image = np.zeros_like(left_image)
    for y in range(height):
        for x in range(width):
            d = int(gt_disparity[y, x])
            if x - d >= 0:
                right_image[y, x - d] = left_image[y, x]
    
    # Fill occlusions in right image
    mask = right_image == 0
    right_image[mask] = np.random.randint(0, 256, np.sum(mask), dtype=np.uint8)
    
    camera_params = {
        'focal_length': focal_length,
        'baseline': baseline,
        'width': width,
        'height': height
    }
    
    return left_image, right_image, gt_disparity, camera_params

# Generate synthetic stereo pair
left_img, right_img, gt_disp, cam_params = generate_synthetic_stereo_pair()

print("✓ Synthetic stereo pair generated!")
print(f"Image dimensions: {left_img.shape}")
print(f"Disparity range: [{gt_disp.min():.2f}, {gt_disp.max():.2f}] pixels")
print(f"Focal length: {cam_params['focal_length']} pixels")
print(f"Baseline: {cam_params['baseline']} cm")

### Visualize Synthetic Data

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(left_img, cmap='gray')
axes[0].set_title('Left Image', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(right_img, cmap='gray')
axes[1].set_title('Right Image', fontsize=14, fontweight='bold')
axes[1].axis('off')

im = axes[2].imshow(gt_disp, cmap='jet')
axes[2].set_title('Ground Truth Disparity', fontsize=14, fontweight='bold')
axes[2].axis('off')
plt.colorbar(im, ax=axes[2], fraction=0.046, pad=0.04, label='Disparity (pixels)')

plt.tight_layout()
plt.show()

print(f"Ground truth disparity statistics:")
print(f"  Mean: {gt_disp.mean():.2f} pixels")
print(f"  Std: {gt_disp.std():.2f} pixels")

## 4. Block Matching Algorithm

Implement traditional block matching using Sum of Squared Differences (SSD) criterion.

### Mathematical Formulation

For each pixel $(x, y)$ in the left image, find disparity $d$ that minimizes:

$$E(d) = \sum_{i,j \in W} \left( I_{left}(x+i, y+j) - I_{right}(x-d+i, y+j) \right)^2$$

where $W$ is the matching window.

In [ ]:
def compute_disparity_block_matching(left_image, right_image, block_size=15, max_disparity=64):
    """
    Compute disparity map using block matching with Sum of Squared Differences (SSD).
    
    Parameters:
    - left_image: Rectified left image (grayscale)
    - right_image: Rectified right image (grayscale)
    - block_size: Size of the matching block (odd number)
    - max_disparity: Maximum disparity to search
    
    Returns:
    - disparity_map: Computed disparity map (pixels)
    """
    height, width = left_image.shape
    disparity_map = np.zeros((height, width), dtype=np.float32)
    half_block = block_size // 2
    
    print(f"Computing disparity using block matching...")
    print(f"  Block size: {block_size}x{block_size}")
    print(f"  Max disparity: {max_disparity}")
    
    start_time = time.time()
    
    for y in range(half_block, height - half_block):
        # Progress indicator
        if y % 50 == 0:
            progress = (y - half_block) / (height - 2 * half_block) * 100
            print(f"  Progress: {progress:.1f}%", end='\r')
        
        for x in range(half_block, width - half_block):
            # Extract left block
            left_block = left_image[y - half_block:y + half_block + 1,
                                   x - half_block:x + half_block + 1]
            
            # Search for best match in right image
            min_ssd = float('inf')
            best_disparity = 0
            
            for d in range(max_disparity + 1):
                if x - d - half_block < 0:
                    continue
                
                # Extract right block
                right_block = right_image[y - half_block:y + half_block + 1,
                                         x - d - half_block:x - d + half_block + 1]
                
                # Compute SSD
                ssd = np.sum((left_block.astype(float) - right_block.astype(float)) ** 2)
                
                if ssd < min_ssd:
                    min_ssd = ssd
                    best_disparity = d
            
            disparity_map[y, x] = best_disparity
    
    elapsed_time = time.time() - start_time
    print(f"\n  Completed in {elapsed_time:.2f} seconds")
    
    return disparity_map

# Compute disparity using block matching
BLOCK_SIZE = 15
MAX_DISPARITY = 64

disparity_bm = compute_disparity_block_matching(left_img, right_img, BLOCK_SIZE, MAX_DISPARITY)

print(f"\n✓ Block matching disparity computed!")
print(f"Disparity statistics:")
print(f"  Mean: {disparity_bm.mean():.2f} pixels")
print(f"  Std: {disparity_bm.std():.2f} pixels")
print(f"  Range: [{disparity_bm.min():.2f}, {disparity_bm.max():.2f}]")

### Visualize Block Matching Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

im1 = axes[0].imshow(gt_disp, cmap='jet')
axes[0].set_title('Ground Truth Disparity', fontsize=14, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04, label='Disparity (pixels)')

im2 = axes[1].imshow(disparity_bm, cmap='jet')
axes[1].set_title('Block Matching Disparity', fontsize=14, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04, label='Disparity (pixels)')

plt.tight_layout()
plt.show()

# Compute error metrics
error_bm = np.abs(disparity_bm - gt_disp)
mae_bm = error_bm.mean()
rmse_bm = np.sqrt((error_bm ** 2).mean())

print(f"\nBlock Matching Error Metrics:")
print(f"  Mean Absolute Error (MAE): {mae_bm:.3f} pixels")
print(f"  Root Mean Square Error (RMSE): {rmse_bm:.3f} pixels")

## 5. Levenberg-Marquardt Optimization

Refine disparity estimates using numerical optimization.

### Levenberg-Marquardt Algorithm

The LM algorithm solves nonlinear least squares problems:

$$\min_d \sum_i r_i(d)^2$$

where $r_i(d) = I_{left}(i) - I_{right}(i-d)$ are residuals.

The update rule is:

$$d_{k+1} = d_k - (J^T J + \lambda I)^{-1} J^T r(d_k)$$

where $J$ is the Jacobian and $\lambda$ is the damping parameter.

In [ ]:
def ssd_residuals(disparity, left_block, right_image, x, y, half_block):
    """
    Compute SSD residuals for Levenberg-Marquardt optimization.
    
    Parameters:
    - disparity: Array with single disparity value
    - left_block: Left image block (flattened)
    - right_image: Right image
    - x, y: Block center coordinates
    - half_block: Half block size
    
    Returns:
    - residuals: Difference between left and right blocks
    """
    d = int(np.clip(disparity[0], 0, 64))
    
    if x - d - half_block < 0 or x - d + half_block + 1 > right_image.shape[1]:
        return np.full_like(left_block, 1000.0)
    
    right_block = right_image[y - half_block:y + half_block + 1,
                              x - d - half_block:x - d + half_block + 1]
    
    residuals = left_block - right_block.flatten().astype(float)
    return residuals


def optimize_disparity_levenberg_marquardt(left_image, right_image, initial_disparity,
                                           block_size=15, num_pixels=500):
    """
    Refine disparity using Levenberg-Marquardt optimization.
    
    Parameters:
    - left_image: Left image (grayscale)
    - right_image: Right image (grayscale)
    - initial_disparity: Initial disparity estimate
    - block_size: Matching block size
    - num_pixels: Number of pixels to optimize
    
    Returns:
    - optimized_disparity: Refined disparity map
    - error_history: Optimization error history
    - convergence_info: Information about convergence
    """
    height, width = left_image.shape
    optimized_disparity = initial_disparity.copy()
    half_block = block_size // 2
    
    print(f"\nOptimizing disparity using Levenberg-Marquardt...")
    print(f"  Number of pixels to optimize: {num_pixels}")
    
    # Select random pixels to optimize
    np.random.seed(42)
    pixels_to_optimize = []
    for _ in range(num_pixels):
        y = np.random.randint(half_block, height - half_block)
        x = np.random.randint(half_block + 64, width - half_block)
        pixels_to_optimize.append((y, x))
    
    all_errors = []
    successful_optimizations = 0
    convergence_messages = []
    
    start_time = time.time()
    
    for idx, (y, x) in enumerate(pixels_to_optimize):
        if idx % 100 == 0:
            progress = idx / len(pixels_to_optimize) * 100
            print(f"  Progress: {progress:.1f}%", end='\r')
        
        # Extract left block
        left_block = left_image[y - half_block:y + half_block + 1,
                               x - half_block:x + half_block + 1].flatten().astype(float)
        
        # Initial guess from block matching
        d0 = initial_disparity[y, x]
        
        # Optimize using Levenberg-Marquardt
        try:
            result = least_squares(
                ssd_residuals,
                [d0],
                args=(left_block, right_image, x, y, half_block),
                method='lm',
                max_nfev=50,
                ftol=1e-6,
                xtol=1e-6
            )
            
            if result.success:
                optimized_disparity[y, x] = np.clip(result.x[0], 0, 64)
                successful_optimizations += 1
                all_errors.append(result.cost)
                convergence_messages.append(result.message)
        except Exception as e:
            pass
    
    elapsed_time = time.time() - start_time
    
    print(f"\n  Completed in {elapsed_time:.2f} seconds")
    print(f"  Successful optimizations: {successful_optimizations}/{num_pixels}")
    print(f"  Success rate: {successful_optimizations/num_pixels*100:.1f}%")
    
    convergence_info = {
        'total': num_pixels,
        'successful': successful_optimizations,
        'time': elapsed_time
    }
    
    return optimized_disparity, all_errors, convergence_info

# Optimize disparity
NUM_OPTIMIZE_PIXELS = 500

disparity_opt, error_history, convergence_info = optimize_disparity_levenberg_marquardt(
    left_img, right_img, disparity_bm, BLOCK_SIZE, NUM_OPTIMIZE_PIXELS
)

print(f"\n✓ Optimization completed!")
print(f"Optimized disparity statistics:")
print(f"  Mean: {disparity_opt.mean():.2f} pixels")
print(f"  Std: {disparity_opt.std():.2f} pixels")
print(f"  Range: [{disparity_opt.min():.2f}, {disparity_opt.max():.2f}]")

### Optimization Convergence Analysis

In [ ]:
# Plot optimization error
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Error distribution
if len(error_history) > 0:
    axes[0].hist(error_history, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Final Cost (SSD)', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('Distribution of Optimization Final Costs', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # Error progression
    axes[1].plot(error_history, 'o-', color='darkred', markersize=3, alpha=0.6)
    axes[1].set_xlabel('Optimization Index', fontsize=12)
    axes[1].set_ylabel('Final Cost (SSD)', fontsize=12)
    axes[1].set_title('Optimization Cost per Pixel', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
else:
    axes[0].text(0.5, 0.5, 'No error data', ha='center', va='center')
    axes[1].text(0.5, 0.5, 'No error data', ha='center', va='center')

plt.tight_layout()
plt.show()

if len(error_history) > 0:
    print(f"\nOptimization Error Statistics:")
    print(f"  Mean final cost: {np.mean(error_history):.2f}")
    print(f"  Median final cost: {np.median(error_history):.2f}")
    print(f"  Std final cost: {np.std(error_history):.2f}")

### Compare Disparity Maps

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

im1 = axes[0].imshow(gt_disp, cmap='jet', vmin=0, vmax=64)
axes[0].set_title('Ground Truth Disparity', fontsize=14, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04, label='Disparity (pixels)')

im2 = axes[1].imshow(disparity_bm, cmap='jet', vmin=0, vmax=64)
axes[1].set_title('Block Matching', fontsize=14, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04, label='Disparity (pixels)')

im3 = axes[2].imshow(disparity_opt, cmap='jet', vmin=0, vmax=64)
axes[2].set_title('Levenberg-Marquardt Optimized', fontsize=14, fontweight='bold')
axes[2].axis('off')
plt.colorbar(im3, ax=axes[2], fraction=0.046, pad=0.04, label='Disparity (pixels)')

plt.tight_layout()
plt.show()

# Compute error metrics for both methods
error_opt = np.abs(disparity_opt - gt_disp)
mae_opt = error_opt.mean()
rmse_opt = np.sqrt((error_opt ** 2).mean())

print(f"\n{'Method':<30} {'MAE (pixels)':<15} {'RMSE (pixels)':<15}")
print(f"{'-'*60}")
print(f"{'Block Matching':<30} {mae_bm:<15.3f} {rmse_bm:<15.3f}")
print(f"{'Levenberg-Marquardt':<30} {mae_opt:<15.3f} {rmse_opt:<15.3f}")
print(f"\nImprovement:")
print(f"  MAE reduction: {(mae_bm - mae_opt)/mae_bm*100:.2f}%")
print(f"  RMSE reduction: {(rmse_bm - rmse_opt)/rmse_bm*100:.2f}%")

## 6. Depth Reconstruction

Convert disparity to depth using triangulation.

### Depth Equation

$$Z = \frac{f \cdot B}{d}$$

where:
- $Z$ = depth (cm)
- $f$ = focal length (pixels)
- $B$ = baseline (cm)
- $d$ = disparity (pixels)

In [ ]:
def disparity_to_depth(disparity, focal_length, baseline):
    """
    Convert disparity map to depth map.
    
    Parameters:
    - disparity: Disparity map (pixels)
    - focal_length: Camera focal length (pixels)
    - baseline: Stereo baseline (cm)
    
    Returns:
    - depth: Depth map (cm)
    """
    # Avoid division by zero
    depth = np.zeros_like(disparity)
    valid = disparity > 0
    depth[valid] = (focal_length * baseline) / disparity[valid]
    
    return depth

# Convert to depth
depth_gt = disparity_to_depth(gt_disp, cam_params['focal_length'], cam_params['baseline'])
depth_bm = disparity_to_depth(disparity_bm, cam_params['focal_length'], cam_params['baseline'])
depth_opt = disparity_to_depth(disparity_opt, cam_params['focal_length'], cam_params['baseline'])

print("✓ Depth maps computed!")
print(f"\nDepth statistics (cm):")
print(f"Ground Truth - Mean: {depth_gt[depth_gt > 0].mean():.2f}, Range: [{depth_gt[depth_gt > 0].min():.2f}, {depth_gt[depth_gt > 0].max():.2f}]")
print(f"Block Matching - Mean: {depth_bm[depth_bm > 0].mean():.2f}, Range: [{depth_bm[depth_bm > 0].min():.2f}, {depth_bm[depth_bm > 0].max():.2f}]")
print(f"Optimized - Mean: {depth_opt[depth_opt > 0].mean():.2f}, Range: [{depth_opt[depth_opt > 0].min():.2f}, {depth_opt[depth_opt > 0].max():.2f}]")

### Visualize Depth Maps

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

im1 = axes[0].imshow(depth_gt, cmap='plasma')
axes[0].set_title('Ground Truth Depth', fontsize=14, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04, label='Depth (cm)')

im2 = axes[1].imshow(depth_bm, cmap='plasma')
axes[1].set_title('Block Matching Depth', fontsize=14, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04, label='Depth (cm)')

im3 = axes[2].imshow(depth_opt, cmap='plasma')
axes[2].set_title('Optimized Depth', fontsize=14, fontweight='bold')
axes[2].axis('off')
plt.colorbar(im3, ax=axes[2], fraction=0.046, pad=0.04, label='Depth (cm)')

plt.tight_layout()
plt.show()

### Depth Error Analysis

In [ ]:
# Compute depth errors
valid_mask = (depth_gt > 0) & (depth_bm > 0) & (depth_opt > 0)

depth_error_bm = np.abs(depth_bm - depth_gt)
depth_error_opt = np.abs(depth_opt - depth_gt)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

im1 = axes[0].imshow(depth_error_bm, cmap='hot', vmin=0, vmax=20)
axes[0].set_title('Block Matching Depth Error', fontsize=14, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04, label='Absolute Error (cm)')

im2 = axes[1].imshow(depth_error_opt, cmap='hot', vmin=0, vmax=20)
axes[1].set_title('Optimized Depth Error', fontsize=14, fontweight='bold')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04, label='Absolute Error (cm)')

plt.tight_layout()
plt.show()

# Statistics
mae_depth_bm = depth_error_bm[valid_mask].mean()
mae_depth_opt = depth_error_opt[valid_mask].mean()
rmse_depth_bm = np.sqrt((depth_error_bm[valid_mask] ** 2).mean())
rmse_depth_opt = np.sqrt((depth_error_opt[valid_mask] ** 2).mean())

print(f"\nDepth Error Metrics (cm):")
print(f"{'Method':<30} {'MAE (cm)':<15} {'RMSE (cm)':<15}")
print(f"{'-'*60}")
print(f"{'Block Matching':<30} {mae_depth_bm:<15.3f} {rmse_depth_bm:<15.3f}")
print(f"{'Levenberg-Marquardt':<30} {mae_depth_opt:<15.3f} {rmse_depth_opt:<15.3f}")

## 7. 3D Point Cloud Reconstruction

Generate 3D point clouds from depth maps.

### 3D Projection

$$X = \frac{(u - c_x) \cdot Z}{f}$$

$$Y = \frac{(v - c_y) \cdot Z}{f}$$

where $(u, v)$ are pixel coordinates and $(c_x, c_y)$ is the principal point.

In [ ]:
def create_point_cloud(depth_map, image, focal_length, subsample=4):
    """
    Create 3D point cloud from depth map.
    
    Parameters:
    - depth_map: Depth map (cm)
    - image: Color/intensity image
    - focal_length: Camera focal length (pixels)
    - subsample: Subsampling factor for efficiency
    
    Returns:
    - points: 3D point coordinates (N x 3)
    - colors: Point colors (N x 3)
    """
    height, width = depth_map.shape
    cx, cy = width / 2, height / 2
    
    points = []
    colors = []
    
    for v in range(0, height, subsample):
        for u in range(0, width, subsample):
            Z = depth_map[v, u]
            
            if Z > 0:  # Valid depth
                X = (u - cx) * Z / focal_length
                Y = (v - cy) * Z / focal_length
                
                points.append([X, Y, Z])
                
                # Color (normalize grayscale to RGB)
                intensity = image[v, u] / 255.0
                colors.append([intensity, intensity, intensity])
    
    return np.array(points), np.array(colors)

# Create point clouds
print("Creating point clouds...")
points_gt, colors_gt = create_point_cloud(depth_gt, left_img, cam_params['focal_length'])
points_bm, colors_bm = create_point_cloud(depth_bm, left_img, cam_params['focal_length'])
points_opt, colors_opt = create_point_cloud(depth_opt, left_img, cam_params['focal_length'])

print(f"✓ Point clouds created!")
print(f"Ground Truth: {len(points_gt)} points")
print(f"Block Matching: {len(points_bm)} points")
print(f"Optimized: {len(points_opt)} points")

### Visualize Point Clouds

In [ ]:
fig = plt.figure(figsize=(20, 6))

# Ground truth
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(points_gt[:, 0], points_gt[:, 1], points_gt[:, 2], 
           c=colors_gt, s=1, alpha=0.5)
ax1.set_xlabel('X (cm)', fontsize=10)
ax1.set_ylabel('Y (cm)', fontsize=10)
ax1.set_zlabel('Z (cm)', fontsize=10)
ax1.set_title('Ground Truth Point Cloud', fontsize=12, fontweight='bold')
ax1.view_init(elev=20, azim=45)

# Block matching
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(points_bm[:, 0], points_bm[:, 1], points_bm[:, 2], 
           c=colors_bm, s=1, alpha=0.5)
ax2.set_xlabel('X (cm)', fontsize=10)
ax2.set_ylabel('Y (cm)', fontsize=10)
ax2.set_zlabel('Z (cm)', fontsize=10)
ax2.set_title('Block Matching Point Cloud', fontsize=12, fontweight='bold')
ax2.view_init(elev=20, azim=45)

# Optimized
ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(points_opt[:, 0], points_opt[:, 1], points_opt[:, 2], 
           c=colors_opt, s=1, alpha=0.5)
ax3.set_xlabel('X (cm)', fontsize=10)
ax3.set_ylabel('Y (cm)', fontsize=10)
ax3.set_zlabel('Z (cm)', fontsize=10)
ax3.set_title('Optimized Point Cloud', fontsize=12, fontweight='bold')
ax3.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

## 8. Performance Analysis

Summary of computational performance and accuracy.

In [ ]:
# Create performance summary table
summary_data = {
    'Metric': [
        'Disparity MAE (pixels)',
        'Disparity RMSE (pixels)',
        'Depth MAE (cm)',
        'Depth RMSE (cm)',
        'Success Rate (%)'
    ],
    'Block Matching': [
        f'{mae_bm:.3f}',
        f'{rmse_bm:.3f}',
        f'{mae_depth_bm:.3f}',
        f'{rmse_depth_bm:.3f}',
        '100.0'
    ],
    'Levenberg-Marquardt': [
        f'{mae_opt:.3f}',
        f'{rmse_opt:.3f}',
        f'{mae_depth_opt:.3f}',
        f'{rmse_depth_opt:.3f}',
        f"{convergence_info['successful']/convergence_info['total']*100:.1f}"
    ]
}

print("\n" + "="*70)
print("PERFORMANCE SUMMARY")
print("="*70)
print(f"\n{'Metric':<30} {'Block Matching':<20} {'L-M Optimized':<20}")
print("-"*70)

for i, metric in enumerate(summary_data['Metric']):
    print(f"{metric:<30} {summary_data['Block Matching'][i]:<20} {summary_data['Levenberg-Marquardt'][i]:<20}")

print("="*70)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Error comparison
methods = ['Block\nMatching', 'Levenberg-\nMarquardt']
mae_values = [mae_bm, mae_opt]
rmse_values = [rmse_bm, rmse_opt]

x = np.arange(len(methods))
width = 0.35

axes[0].bar(x - width/2, mae_values, width, label='MAE', color='steelblue', alpha=0.8)
axes[0].bar(x + width/2, rmse_values, width, label='RMSE', color='coral', alpha=0.8)
axes[0].set_ylabel('Error (pixels)', fontsize=12)
axes[0].set_title('Disparity Error Comparison', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(methods)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Depth error comparison
depth_mae_values = [mae_depth_bm, mae_depth_opt]
depth_rmse_values = [rmse_depth_bm, rmse_depth_opt]

axes[1].bar(x - width/2, depth_mae_values, width, label='MAE', color='steelblue', alpha=0.8)
axes[1].bar(x + width/2, depth_rmse_values, width, label='RMSE', color='coral', alpha=0.8)
axes[1].set_ylabel('Error (cm)', fontsize=12)
axes[1].set_title('Depth Error Comparison', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(methods)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 9. Conclusions

### Key Findings

1. **Block Matching**: Provides fast, reasonable initial estimates but suffers from noise and ambiguities in textureless regions.

2. **Levenberg-Marquardt Optimization**: Successfully refines disparity estimates through nonlinear least squares optimization, improving accuracy while maintaining computational efficiency.

3. **Depth Reconstruction**: Accurate depth maps enable realistic 3D point cloud generation for VR applications.

4. **Numerical Analysis**: The project demonstrates practical applications of numerical optimization in computer vision, showing how mathematical methods can enhance algorithmic performance.

### Future Work

- Implement GPU acceleration for real-time performance
- Explore other optimization algorithms (Gauss-Newton, Trust Region)
- Add edge-preserving smoothness constraints
- Test on real stereo datasets (KITTI, Middlebury)

---

**End of Notebook**